In [2]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import os
import json
import time
from itertools import chain


#!conda config --env --add channels conda-forge
#!conda install numpy -y
#!conda install pandas -y
#!conda install matplotlib -y

-  Spotify Million Playlist Dataset Challenge https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#dataset
- How to Develop an End-to-End Machine Learning Project and Deploy it to Heroku with Flask https://www.freecodecamp.org/news/end-to-end-machine-learning-project-turorial/
- A Full End-to-End Deployment of a Machine Learning Algorithm into a Live Production Environment https://towardsdatascience.com/a-full-end-to-end-deployment-of-a-machine-learning-algorithm-into-a-live-production-environment-3d9971ade188
- Object Oriented Machine Learning — Create Production grade Machine Learning Pipeline using OOPs https://medium.com/analytics-vidhya/object-oriented-machine-learning-create-production-grade-machine-learning-pipeline-using-oops-50f3cfa14659

In [64]:
%time

playlist = pd.DataFrame()
track = pd.DataFrame()
playlist_track = pd.DataFrame()

path = "/home/maksim/Documents/ML/Datasets/Spotify_data/"
filenames = os.listdir(path)
for filename in filenames[:5]:
    fullpath = os.sep.join((path, filename))
    f = open(fullpath)
    js = f.read()
    f.close()
    mpd_slice = json.loads(js)
    playlist_temp = pd.DataFrame(mpd_slice['playlists'])
    track_temp = pd.DataFrame(list(chain.from_iterable(playlist_temp['tracks'])))
    playlist_track_temp = pd.DataFrame()
    
    track = pd.concat([track, track_temp]).drop_duplicates()
    playlist = pd.concat([playlist, playlist_temp])
    playlist_track = pd.concat([playlist_track, pd.DataFrame()])

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


In [138]:
index_pos_0 = pd.Series(track_temp.loc[track_temp.pos == 0, 'pos'].index)

times = index_pos_0.diff()[1:].reset_index(drop=True).astype(int)

temp_uid = [[playlist_temp.pid[i]]*times[i] for i in range(999)]
temp_uid.append([playlist_temp.pid[999]]*(track_temp.index.max() - index_pos_0.max()))

In [142]:
temp_uid = np.array(temp_uid).reshape([-1, 1])
temp_uid

array([[list([477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000, 477000])],
       [list([477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001, 477001])],
       [list([477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 477002, 

In [88]:
times = pd.Series(track_temp.loc[track_temp.pos == 0, 'pos'].index).diff()[1:].reset_index(drop=True)
times

0       57.0
1       15.0
2      102.0
3       35.0
4       32.0
       ...  
994     20.0
995     22.0
996     64.0
997     42.0
998     46.0
Length: 999, dtype: float64

122

In [80]:
track_temp.loc[track_temp.pos == 0, 'pos']

0        0
57       0
72       0
174      0
209      0
        ..
66209    0
66231    0
66295    0
66337    0
66383    0
Name: pos, Length: 1000, dtype: int64

In [ ]:
track_temp.loc[track_temp.pos == 0, 'pos']

Series([], Name: pos, dtype: float64)

In [56]:
track_temp

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,768000.0,The Weeknd,spotify:track:6u0dQik0aif7FQlrhycG1L,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,The Morning,spotify:album:5EbpxRwbbpCJUepbqVTZ1U,312426,Trilogy
1,1.0,Wale,spotify:track:3MAgQuClHcAV8E9CbeBS6f,spotify:artist:67nwj3Y5sZQLl72VNUHEYE,Lotus Flower Bomb - feat. Miguel,spotify:album:0jCVC8ndYYOooEY2YTO1l6,213253,Ambition
2,2.0,A$AP Rocky,spotify:track:4S7YHmlWwfwArgd8LfSPud,spotify:artist:13ubrt8QOOCPljQ2FL1Kca,L$D,spotify:album:3arNdjotCvtiiLFfjKngMc,238306,AT.LONG.LAST.A$AP
3,3.0,Rick Ross,spotify:track:6adehXz8vMUmXWJhEIdE6e,spotify:artist:1sBkRIssrMs1AbVkOJbc7a,Sorry,spotify:album:5OQ46UiKltoHtHrGFXB1HI,330506,Black Market
4,4.0,J. Cole,spotify:track:4gkl7QbQvBd3TM9PhlWFkI,spotify:artist:6l3HvQ5sa6mXTsMTB19rO5,Love Yourz,spotify:album:7viNUmZZ8ztn2UB4XB3jIL,211680,2014 Forest Hills Drive
...,...,...,...,...,...,...,...,...
65169,4.0,Florence + The Machine,spotify:track:2ZdWa7qK1fe0dUSP6mJj4e,spotify:artist:1moxjboGR7GNWYIMWsRjgG,My Boy Builds Coffins,spotify:album:2FgknX5e7fJlriQtxvpLhZ,176160,Lungs
65170,5.0,Solange,spotify:track:7zbX1txfcs42K2PlhFJNNE,spotify:artist:2auiVi8sUZo17dLy1HwrTU,Weary,spotify:album:3Yko2SxDk4hc6fncIBQlcM,194253,A Seat at the Table
65171,6.0,Amy Winehouse,spotify:track:7DANYWY97PiBIEzVQRnQfh,spotify:artist:6Q192DXotxtaysaqNPy5yR,Some Unholy War,spotify:album:6GJCGWfI95aeRsdtVB52vc,141266,Back To Black
65172,7.0,Marvin Gaye,spotify:track:27m1soUndRthrAA1ediOXn,spotify:artist:3koiLjNrgRTNbOwViDipeA,I Heard It Through The Grapevine,spotify:album:1jcnZvZWvAGzNyQ7GNVy8X,195266,I Heard It Through The Grapevine / In The Groove


In [29]:
playlist_temp

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Movies,false,768000,1416700800,24,20,1,"[{'pos': 0, 'artist_name': 'The Neighbourhood'...",3,5735549,16,NaN
1,Road Trip Playlist,false,768001,1377475200,40,39,1,"[{'pos': 0, 'artist_name': 'Santigold', 'track...",2,9512398,40,NaN
2,Study,false,768002,1509062400,22,22,1,"[{'pos': 0, 'artist_name': 'Royal Blood', 'tra...",8,4928679,18,NaN
3,fast and furious,false,768003,1490486400,18,17,1,"[{'pos': 0, 'artist_name': 'DJ Shadow', 'track...",4,3986886,11,NaN
4,beach bum,false,768004,1500940800,40,33,1,"[{'pos': 0, 'artist_name': 'Clean Bandit', 'tr...",19,8805418,24,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Playlist,false,768995,1479254400,36,30,1,"[{'pos': 0, 'artist_name': 'Kanye West', 'trac...",9,8015404,26,NaN
996,singles,false,768996,1506211200,94,88,1,"[{'pos': 0, 'artist_name': 'The Verve', 'track...",62,22549092,79,NaN
997,Electronic,false,768997,1480032000,90,72,1,"[{'pos': 0, 'artist_name': 'Rusko', 'track_uri...",48,23120560,49,NaN
998,summer 2015,false,768998,1442448000,22,22,12,"[{'pos': 0, 'artist_name': 'GAMPER & DADONI', ...",10,5323759,22,NaN


In [28]:
track_temp.pos

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,The Weeknd,spotify:track:6u0dQik0aif7FQlrhycG1L,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,The Morning,spotify:album:5EbpxRwbbpCJUepbqVTZ1U,312426,Trilogy
1,1,Wale,spotify:track:3MAgQuClHcAV8E9CbeBS6f,spotify:artist:67nwj3Y5sZQLl72VNUHEYE,Lotus Flower Bomb - feat. Miguel,spotify:album:0jCVC8ndYYOooEY2YTO1l6,213253,Ambition
2,2,A$AP Rocky,spotify:track:4S7YHmlWwfwArgd8LfSPud,spotify:artist:13ubrt8QOOCPljQ2FL1Kca,L$D,spotify:album:3arNdjotCvtiiLFfjKngMc,238306,AT.LONG.LAST.A$AP
3,3,Rick Ross,spotify:track:6adehXz8vMUmXWJhEIdE6e,spotify:artist:1sBkRIssrMs1AbVkOJbc7a,Sorry,spotify:album:5OQ46UiKltoHtHrGFXB1HI,330506,Black Market
4,4,J. Cole,spotify:track:4gkl7QbQvBd3TM9PhlWFkI,spotify:artist:6l3HvQ5sa6mXTsMTB19rO5,Love Yourz,spotify:album:7viNUmZZ8ztn2UB4XB3jIL,211680,2014 Forest Hills Drive
...,...,...,...,...,...,...,...,...
65169,4,Florence + The Machine,spotify:track:2ZdWa7qK1fe0dUSP6mJj4e,spotify:artist:1moxjboGR7GNWYIMWsRjgG,My Boy Builds Coffins,spotify:album:2FgknX5e7fJlriQtxvpLhZ,176160,Lungs
65170,5,Solange,spotify:track:7zbX1txfcs42K2PlhFJNNE,spotify:artist:2auiVi8sUZo17dLy1HwrTU,Weary,spotify:album:3Yko2SxDk4hc6fncIBQlcM,194253,A Seat at the Table
65171,6,Amy Winehouse,spotify:track:7DANYWY97PiBIEzVQRnQfh,spotify:artist:6Q192DXotxtaysaqNPy5yR,Some Unholy War,spotify:album:6GJCGWfI95aeRsdtVB52vc,141266,Back To Black
65172,7,Marvin Gaye,spotify:track:27m1soUndRthrAA1ediOXn,spotify:artist:3koiLjNrgRTNbOwViDipeA,I Heard It Through The Grapevine,spotify:album:1jcnZvZWvAGzNyQ7GNVy8X,195266,I Heard It Through The Grapevine / In The Groove


In [27]:
pd.concat([playlist_temp.pid, track_temp.track_uri ], axis = 1)

,pid,track_uri
0,768000.0,spotify:track:6u0dQik0aif7FQlrhycG1L
1,768001.0,spotify:track:3MAgQuClHcAV8E9CbeBS6f
2,768002.0,spotify:track:4S7YHmlWwfwArgd8LfSPud
3,768003.0,spotify:track:6adehXz8vMUmXWJhEIdE6e
4,768004.0,spotify:track:4gkl7QbQvBd3TM9PhlWFkI
...,...,...
65169,NaN,spotify:track:2ZdWa7qK1fe0dUSP6mJj4e
65170,NaN,spotify:track:7zbX1txfcs42K2PlhFJNNE
65171,NaN,spotify:track:7DANYWY97PiBIEzVQRnQfh
65172,NaN,spotify:track:27m1soUndRthrAA1ediOXn


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [193]:
t = pd.DataFrame()
for i in range(len(mpd_df['tracks'])):
    t = pd.concat([t, pd.DataFrame(mpd_df['tracks'].iloc[i])])

In [194]:
t.track_uri.unique().size

33818

In [201]:
list()

ValueError: If using all scalar values, you must pass an index

In [114]:

pd.json_normalize(mpd_slice['playlists'])['tracks']

pd.DataFrame(df.content.str[0].tolist()).set_index(df.date_hour)



0      [{'pos': 0, 'artist_name': 'Kaya Stewart', 'tr...
1      [{'pos': 0, 'artist_name': 'Evanescence', 'tra...
2      [{'pos': 0, 'artist_name': 'Gorillaz', 'track_...
3      [{'pos': 0, 'artist_name': 'Rihanna', 'track_u...
4      [{'pos': 0, 'artist_name': 'Shakey Graves', 't...
                             ...                        
995    [{'pos': 0, 'artist_name': 'Nu', 'track_uri': ...
996    [{'pos': 0, 'artist_name': 'Midnite String Qua...
997    [{'pos': 0, 'artist_name': 'Carlos Vives', 'tr...
998    [{'pos': 0, 'artist_name': 'Ratatat', 'track_u...
999    [{'pos': 0, 'artist_name': 'Newsboys', 'track_...
Name: tracks, Length: 1000, dtype: object

In [30]:
df_playlists.columns

Index(['name', 'collaborative', 'pid', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms',
       'num_artists', 'description'],
      dtype='object')

In [17]:
df_playlists = pd.DataFrame(mpd_slice['playlists'])


,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,england,false,578000,1501891200,78,62,1,"[{'pos': 0, 'artist_name': 'Kaya Stewart', 'tr...",2,17384475,62,NaN
1,Brett,false,578001,1502928000,108,93,4,"[{'pos': 0, 'artist_name': 'Evanescence', 'tra...",41,25941586,74,NaN
2,Party,false,578002,1369785600,83,73,1,"[{'pos': 0, 'artist_name': 'Gorillaz', 'track_...",5,19892048,54,NaN
3,sex,false,578003,1507593600,23,22,1,"[{'pos': 0, 'artist_name': 'Rihanna', 'track_u...",11,5042248,12,NaN
4,Saturday,false,578004,1509148800,155,137,1,"[{'pos': 0, 'artist_name': 'Shakey Graves', 't...",71,33463729,123,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,dance,false,578995,1508716800,38,33,1,"[{'pos': 0, 'artist_name': 'Nu', 'track_uri': ...",21,11358265,34,NaN
996,Study Hall,false,578996,1503532800,76,23,1,"[{'pos': 0, 'artist_name': 'Midnite String Qua...",3,17919019,9,NaN
997,Road Trip,false,578997,1467244800,117,96,2,"[{'pos': 0, 'artist_name': 'Carlos Vives', 'tr...",2,27944984,67,NaN
998,go hard,true,578998,1370217600,173,107,3,"[{'pos': 0, 'artist_name': 'Ratatat', 'track_u...",35,49690582,62,NaN


In [24]:
pd.DataFrame(df_playlists.tracks[0])

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Kaya Stewart,spotify:track:1u2tUlfNU6ncpaooRo7GN6,spotify:artist:0ay2fTD85rNUhJz8rlxE7a,In Love With a Boy,spotify:album:7qGZ5CPGP7KPDRMa98PcEj,173080,Kaya Stewart
1,1,Kelly Clarkson,spotify:track:0yUlMnLpU2W6JQtvg1k4Od,spotify:artist:3BmGtnKgCSGYIUhmivXKWX,Invincible,spotify:album:7oKtXc3FkeOZTCB88YugON,238360,Piece By Piece (Deluxe Version)
2,2,Ed Sheeran,spotify:track:0A2J5TumCpT4aJVvQHNEQW,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,One,spotify:album:6NoBzYmh5gUusGPCfg0pct,252760,x
3,3,Ed Sheeran,spotify:track:3KiktyOxQd2Z0bmnabgRq2,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,I'm A Mess,spotify:album:6NoBzYmh5gUusGPCfg0pct,244573,x
4,4,Ed Sheeran,spotify:track:6u0oHknFYrnUd42A6rvFly,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,Sing,spotify:album:6NoBzYmh5gUusGPCfg0pct,235382,x
...,...,...,...,...,...,...,...,...
73,73,Echosmith,spotify:track:3WKg25vrbjJlkhsgl2W4p3,spotify:artist:1PbBg2aYjWLKRk84zJK15x,Bright,spotify:album:1oHY6eQmEG8skElDvFgKz2,220626,Talking Dreams
74,74,George Ezra,spotify:track:7GJClzimvMSghjcrKxuf1M,spotify:artist:2ysnwxxNtSgbb9t1m2Ur4j,Budapest,spotify:album:5tF2lAa2rh2kU2xIiBzWia,200720,Wanted On Voyage
75,75,The Deep Dixie Boys,spotify:track:6lXyofHYyEO0hRu8YYKmaL,spotify:artist:7zsszvWNKWDrFOLt2tLC5u,Friends in Low Places,spotify:album:4JpPMfD1WkIJtmLTsfcOkJ,229956,Only a Country Heart
76,76,Katy Tiz,spotify:track:5zwwXMjE6OrqfVpYGnA9RW,spotify:artist:2E19mfEFhCr6UgZUYJGOEW,Whistle (While You Work It),spotify:album:1C0txQpTvo2z5N6Rrm6Y9D,215032,Whistle (While You Work It)


In [70]:
def process_playlists(path):
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice["playlists"]:
                print_playlist(playlist)
                
def print_playlist(playlist):
    print("=====", playlist["pid"], "====")
    print("name:          ", playlist["name"])
    ts = time.strftime(
        "%Y-%m-%d %H:%M:%S", time.localtime(playlist["modified_at"])
    )

    print("last_modified: ", ts)
    print("num edits: ", playlist["num_edits"])
    print("num followers: ", playlist["num_followers"])
    print("num artists: ", playlist["num_artists"])
    print("num albums: ", playlist["num_albums"])
    print("num tracks: ", playlist["num_tracks"])
    print()
    for i, track in enumerate(playlist["tracks"]):
        print(
            "   %3d %s - %s - %s"
            % (i + 1, track["track_name"], track["album_name"], track["artist_name"])
        )
    print()


In [79]:
print_playlist(df_playlists.loc[3])

===== 578003 ====
name:           sex
last_modified:  2017-10-10 03:00:00
num edits:  11
num followers:  1
num artists:  12
num albums:  22
num tracks:  23

     1 Watch n' Learn - Talk That Talk - Rihanna
     2 Cockiness (Love It) - Talk That Talk - Rihanna
     3 Birthday Cake - Talk That Talk - Rihanna
     4 Raining Men - Loud - Rihanna
     5 Planez - Late Nights: The Album - Jeremih
     6 BLOW MY LOAD - Cherry Bomb - Tyler, The Creator
     7 Lotus Flower Bomb - feat. Miguel - Ambition - Wale
     8 Dance for You - 4 - Beyoncé
     9 the valley - Wildheart - Miguel
    10 Pussy is Mine - Kaleidoscope Dream (Deluxe Version) - Miguel
    11 Bad - Remix feat. Rihanna - The Gifted - Wale
    12 Neighbors Know My Name - Ready - Trey Songz
    13 Slow Motion - Trigga Reloaded - Trey Songz
    14 B.E.D. - B.E.D. - Jacquees
    15 Come Thru - 19 - EP - Jacquees
    16 Jump - Unapologetic - Rihanna
    17 Nobody Else But You - Tremaine The Album - Trey Songz
    18 Wild Thoughts - Grate

### 